In [87]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cityblock, euclidean
import random

In [88]:
class Detector:
     
    def __init__(self, subjects):
        self.user_scores = []
        self.imposter_scores = []
        self.mean_vector = []
        self.subjects = subjects
        self.scores = []
        
    def training(self):
        pass
    
    def testing(self):
        pass

    def func(self):
        pass
    
    
    def clasify(self, subject, vector):
        self.vector = vector
        genuine_data = data.loc[data.subject == subject, "H.period":"H.Return"]
        imposter_data = data.loc[data.subject != subject, :]
        self.train = genuine_data[:300]
        self.test_genuine = genuine_data[300:]
        self.test_imposter = imposter_data.groupby("subject").head(2).loc[:, "H.period":"H.Return"]
        self.training()
        self.testing()
        self.func()
        print("gen: "+str(np.mean(self.user_scores))+" "+str(min(self.user_scores)))
        print("scr: "+str(np.mean(self.scores))+" "+str(min(self.scores)))
        print("imp: "+str(np.mean(self.imposter_scores))+" "+str(min(self.imposter_scores)))
        print("std: "+str(np.std(self.user_scores))+" "+str(np.std(self.imposter_scores)))
        print(abs(np.mean(self.scores)-np.mean(self.user_scores)))
        print(abs(np.mean(self.scores)-np.mean(self.imposter_scores)))
        return abs(np.mean(self.scores)-np.mean(self.user_scores))<np.std(self.user_scores) and abs(np.mean(self.scores)-np.mean(self.imposter_scores))>np.std(self.user_scores)*3 and abs(min(self.scores)-min(self.user_scores))<np.std(self.user_scores)


In [107]:
class NearestNeighbourMahalanobisDetector(Detector):
        
        
    def training(self):
        self.cov = np.linalg.inv(np.cov(self.train.T))
    

    def testing(self):
        self.genuine_scores = []
        self.imposter_scores = []
        for i in range(self.test_genuine.shape[0]):
            scores = []
            for j in range(self.train.shape[0]):
                sub = self.test_genuine.iloc[i].values - self.train.iloc[j].values
                dist = np.dot(np.dot(sub.T, self.cov), sub)
                scores.append(dist)
            del scores[scores.index(min(scores))]
            self.user_scores.append(min(scores))

        for i in range(self.test_imposter.shape[0]):
            scores = []
            for j in range(self.train.shape[0]):
                sub = self.test_imposter.iloc[i].values - self.train.iloc[j].values
                dist = np.dot(np.dot(sub.T, self.cov), sub)
                scores.append(dist)
            del scores[scores.index(min(scores))]
            self.imposter_scores.append(min(scores))

    def func(self):
        self.scores =[]
        for j in range(self.train.shape[0]):
            sub = self.vector - self.train.iloc[j].values
            dist = np.dot(np.dot(sub.T, self.cov), sub)
            self.scores.append(dist)
        del self.scores[self.scores.index(min(self.scores))]
        del self.scores[self.scores.index(max(self.scores))]
        

In [108]:
class ManhattanDetector(Detector):
    
    def training(self):
        self.mean_vector = self.train.mean().values         
 
    def testing(self):
        for i in range(self.test_genuine.shape[0]):
            cur_score = cityblock(self.test_genuine.iloc[i].values, \
                                   self.mean_vector)
            self.user_scores.append(cur_score)
 
        for i in range(self.test_imposter.shape[0]):
            cur_score = cityblock(self.test_imposter.iloc[i].values, \
                                   self.mean_vector)
            self.imposter_scores.append(cur_score)
    def func(self):
        self.scores = [cityblock(self.vector, self.mean_vector)]

In [114]:
class ManhattanFilteredDetector(Detector):
    
    def training(self):
        self.mean_vector = self.train.mean().values
        print(self.mean_vector)
        self.std_vector = self.train.std().values
        dropping_indices = []
        for i in range(self.train.shape[0]):
            cur_score = euclidean(self.train.iloc[i].values, 
                                   self.mean_vector)
            if (cur_score > 3*self.std_vector).all() == True:
                dropping_indices.append(i)
        self.train = self.train.drop(self.train.index[dropping_indices])
        self.mean_vector = self.train.mean().values

    def testing(self):
        for i in range(self.test_genuine.shape[0]):
            cur_score = cityblock(self.test_genuine.iloc[i].values, \
                                   self.mean_vector)
            self.user_scores.append(cur_score)
 
        for i in range(self.test_imposter.shape[0]):
            cur_score = cityblock(self.test_imposter.iloc[i].values, \
                                   self.mean_vector)
            self.imposter_scores.append(cur_score)
    def func(self):
        self.scores = [cityblock(self.vector, self.mean_vector)]